# Deep Learning Model with dynamic Between - Class examples

### In this Notebook x_train examples shuffle and combine efery second epoch 

In [3]:
import numpy as np
import tensorflow as tf
import keras
import sklearn
import json
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adamax
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.regularizers import l2
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score
from keras.layers.normalization import BatchNormalization

np.random.seed(1)
tf.set_random_seed(1)
bands = 75
frames = 75
num_channels = 3
classes = 8

Using TensorFlow backend.


Function **do_scale (x4d, scaler_list)**
 scales channels of samples with list of sklearn **StandartScalers**

In [4]:
def do_scale(x4d, scaler_list):
    (n_clips, n_time, n_freq, n_channel) = x4d.shape
    x4d_scaled = np.zeros(x4d.shape)

    for channel in range(n_channel):
        x2d = x4d[:,:,:,channel].reshape((n_clips * n_time, n_freq))
        x2d_scaled = scaler_list[channel].transform(x2d)
        x3d_scaled = x2d_scaled.reshape((n_clips, n_time, n_freq))
        x4d_scaled[:,:,:,channel] = x3d_scaled

    return x4d_scaled

Function **preprocess_data(x_train_path, y_train_path,x_test_path, y_test_path)**:
1. Load data from .npy
2. Fit Scalers on train data
3. Remove unknown samples from test
4. Transform y

In [10]:
def preprocess_data(x_train_path, y_train_path,x_test_path, y_test_path):
    x_train = np.load(x_train_path).astype(np.float32)
    y_train = np.load(y_train_path).astype(np.float32)
    x_test = np.load(x_test_path).astype(np.float32)
    y_test = np.load(y_test_path).astype(np.float32)

    scaler_list = []
    (n_clips, n_time, n_freq, n_channel) = x_train.shape

    for channel in range(n_channel):
        xtrain_2d = x_train[:, :, :, channel].reshape((n_clips * n_time, n_freq))
        scaler = sklearn.preprocessing.StandardScaler().fit(xtrain_2d)
        scaler_list += [scaler]

    x_train = do_scale(x_train, scaler_list)
    np.save('test_scaled.npy',do_scale(x_test, scaler_list))
    x_test = do_scale(x_test, scaler_list)[:y_test.shape[0]-137, :, :, :]
    y_train = keras.utils.to_categorical(y_train,classes)
    y_test = y_test[:y_test.shape[0]-137]
    y_test = keras.utils.to_categorical(y_test,classes)
    return x_train, y_train, x_test, y_test

### Building model

In [11]:
def build_model():
    model = Sequential()

    # section 1

    model.add(Convolution2D(filters=32, kernel_size=5,
                            strides=2,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal",
                            input_shape=(frames, bands, num_channels)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=32, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.3))

    # section 2
    model.add(Convolution2D(filters=64, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=64, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # section 3
    model.add(Convolution2D(filters=128, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=128, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=128, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(Convolution2D(filters=128, kernel_size=3,
                            strides=1,
                            padding="same",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    # section 4
    model.add(Convolution2D(filters=512, kernel_size=3,
                            strides=1,
                            padding="valid",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    model.add(Convolution2D(filters=512, kernel_size=1,
                            strides=1,
                            padding="valid",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    # section 5
    model.add(Convolution2D(filters=8, kernel_size=1,
                            strides=1,
                            padding="valid",
                            kernel_regularizer=l2(0.0001),
                            kernel_initializer="normal"))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(GlobalAveragePooling2D())

    model.add(Activation('softmax'))

    return model

Function **shuffle_in_unison(a, b)** mix x with y

In [12]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b

Function **combine_samples(x1,y1,x2,y2)** combine 2 samples with random ratio

In [13]:
def combine_samples(x1,y1,x2,y2):
    combine_k = np.random.random_sample()
    x_new = combine_k*x1+(1-combine_k)*x2
    y_new = combine_k*y1+(1-combine_k)*y2
    return x_new, y_new

In [14]:
x_train, y_train, x_test, y_test = preprocess_data('train_features.npy',
                                                   'train_labels.npy',
                                                   'test_features.npy',
                                                   'test_labels.npy')

x_train, y_train = shuffle_in_unison(x_train, y_train)

reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, min_lr=1e-7)

early_stopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)
callbacks = [reduce_lr_on_plateau, early_stopping]
acc_list = []

model = build_model()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=Adamax(0.01))
json_model = json.loads(model.to_json())
with open('model.json', 'w') as outfile:
    json.dump(json_model, outfile)

In [ ]:
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=callbacks,
                        batch_size=256,
                        epochs=1)
model.save_weights('weights/regular/model_weights1.hdf5')
history2 = model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=callbacks,
                        batch_size=256,
                        epochs=1)
model.save_weights('weights/regular/model_weights2.hdf5')
history3 = model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=callbacks,
                        batch_size=256,
                        epochs=20)
model.save_weights('weights/regular/model_weights20.hdf5')
history4 = model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=callbacks,
                        batch_size=256,
                        epochs=10)
model.save_weights('weights/regular/model_weights30.hdf5')
history5 = model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=callbacks,
                        batch_size=256,
                        epochs=10)
model.save_weights('weights/regular/model_weights40.hdf5')


Unfortunately my machine does not allow me to fit the model fast, so the fitting is not complete and a model with a smaller val_loss is chosen.

In [11]:
model.load_weights('weights/regular/model_weights40.hdf5')
print(model.test_on_batch(x_test,y_test))

[0.4281749, 0.92389005]


I do not understand, how SQ metric works, so i haven't tested this metric.